In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report, accuracy_score
import numpy as np


In [ ]:
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 1.6.1.


In [ ]:
df = pd.read_csv("insurance.csv")

In [ ]:
df.sample(5)

,age,weight,height,income_lpa,smoker,city,occupation,insurance_premium_category
46,42,83.0,1.57,25.57,True,Kolkata,unemployed,High
26,33,79.0,1.61,23.61,False,Jaipur,freelancer,Medium
42,23,69.9,1.79,2.60,True,Mysore,student,Medium
32,47,113.7,1.90,50.00,False,Jalandhar,private_job,Medium
92,37,62.7,1.85,30.00,True,Lucknow,government_job,Low


In [ ]:
df.sample(5)

,age,weight,height,income_lpa,smoker,city,occupation,insurance_premium_category
20,34,58.2,1.85,30.650000,True,Gaya,business_owner,Medium
91,38,119.8,1.76,28.467885,False,Bangalore,government_job,Low
24,50,54.2,1.66,18.600000,False,Mysore,private_job,Medium
64,71,117.8,1.78,1.020000,False,Chandigarh,retired,High
8,73,58.0,1.58,1.780000,False,Chandigarh,retired,Medium


In [ ]:
df['occupation'].unique()

array(['retired', 'freelancer', 'student', 'government_job',
       'business_owner', 'unemployed', 'private_job'], dtype=object)

In [ ]:
df_feat = df.copy()

In [ ]:
df_feat['bmi'] = df_feat['weight']/(df_feat['height']**2)

In [ ]:
def age_group(age):
  if age < 25:
    return "young"
  elif age<45:
    return "adult"
  elif age<60:
    return "middle-aged"
  return "senior"


In [ ]:
df_feat['age_grouped'] = df_feat['age'].apply(age_group)

In [ ]:
def lifestyle_risk(row):
  if row['smoker'] and  row["bmi"]>30:
    return "high"
  elif row["smoker"] and row["bmi"]>27:
    return "medium"
  else:
    return "low"


In [ ]:
df_feat['lifestyle_risk'] = df_feat.apply(lifestyle_risk,axis=1)

In [ ]:
tier_1_cities = ["Mumbai", "Delhi", "Bangalore", "Chennai", "Kolkata", "Hyderabad", "Pune"]
tier_2_cities = [
    "Jaipur", "Chandigarh", "Indore", "Lucknow", "Patna", "Ranchi", "Visakhapatnam", "Coimbatore",
    "Bhopal", "Nagpur", "Vadodara", "Surat", "Rajkot", "Jodhpur", "Raipur", "Amritsar", "Varanasi",
    "Agra", "Dehradun", "Mysore", "Jabalpur", "Guwahati", "Thiruvananthapuram", "Ludhiana", "Nashik",
    "Allahabad", "Udaipur", "Aurangabad", "Hubli", "Belgaum", "Salem", "Vijayawada", "Tiruchirappalli",
    "Bhavnagar", "Gwalior", "Dhanbad", "Bareilly", "Aligarh", "Gaya", "Kozhikode", "Warangal",
    "Kolhapur", "Bilaspur", "Jalandhar", "Noida", "Guntur", "Asansol", "Siliguri"
]

In [ ]:
def city_tier(city):
  if city in tier_1_cities:
    return 1
  elif city in tier_2_cities:
    return 2
  else:
    return 3


In [ ]:
df_feat['city_tier'] = df_feat['city'].apply(city_tier)

In [ ]:
df_feat.drop(columns=['age', 'weight', 'height', 'smoker', 'city'])[['income_lpa', 'occupation', 'bmi', 'age_grouped', 'lifestyle_risk', 'city_tier', 'insurance_premium_category']].sample(5)

,income_lpa,occupation,bmi,age_grouped,lifestyle_risk,city_tier,insurance_premium_category
85,34.66,private_job,14.857209,adult,low,1,Low
0,2.92,retired,49.227482,senior,low,2,High
68,0.68,student,22.963196,Young,low,2,Low
54,3.32,retired,21.025423,senior,low,2,High
48,32.69,unemployed,34.402671,adult,high,1,Medium


In [ ]:
# Select features and target
X = df_feat[["bmi", "age_grouped", "lifestyle_risk", "city_tier", "income_lpa", "occupation"]]
y = df_feat["insurance_premium_category"]

In [ ]:
X

,bmi,age_grouped,lifestyle_risk,city_tier,income_lpa,occupation
0,49.227482,senior,low,2,2.92000,retired
1,30.189017,adult,low,1,34.28000,freelancer
2,21.118382,adult,low,2,36.64000,freelancer
3,45.535900,Young,high,1,3.34000,student
4,24.296875,senior,low,2,3.94000,retired
...,...,...,...,...,...,...
95,21.420747,adult,low,2,19.64000,business_owner
96,47.984483,adult,low,1,34.01000,private_job
97,18.765432,middle-aged,low,1,44.86000,freelancer
98,30.521676,adult,low,1,28.30000,business_owner


In [ ]:
y

,insurance_premium_category
0,High
1,Low
2,Low
3,Medium
4,High
...,...
95,Low
96,Low
97,Low
98,Low


In [ ]:
# Define categorical and numeric features
categorical_features = ["age_grouped", "lifestyle_risk", "occupation", "city_tier"]
numeric_features = ["bmi", "income_lpa"]

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(), categorical_features),
        ("num", "passthrough", numeric_features)
    ]
)

In [ ]:
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", RandomForestClassifier(random_state=42))
])

In [ ]:
# Split data and train model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat', OneHotEncoder(),
                                                  ['age_grouped',
                                                   'lifestyle_risk',
                                                   'occupation', 'city_tier']),
                                                 ('num', 'passthrough',
                                                  ['bmi', 'income_lpa'])])),
                ('classifier', RandomForestClassifier(random_state=42))])

In [ ]:
# Predict and evaluate
y_pred = pipeline.predict(X_test)
accuracy_score(y_test, y_pred)


0.8

In [ ]:
X_test.sample(5)

,bmi,age_grouped,lifestyle_risk,city_tier,income_lpa,occupation
10,22.949982,adult,low,1,32.78,business_owner
92,18.319942,adult,low,2,30.00,government_job
31,15.258742,adult,low,2,11.77,private_job
84,28.801497,senior,medium,2,0.62,retired
44,30.078125,middle-aged,high,2,50.00,private_job


In [ ]:
import pickle

pickle_model_path = "model.pkl"
with open(pickle_model_path, "wb") as f:
  pickle.dump(pipeline,f)